In [ ]:
import pandas as pd
import requests
from io import StringIO
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Point
from shapely.geometry import GeometryCollection, MultiPolygon, Polygon, mapping
import os
import pandas_gbq
import shapely
from google.oauth2.service_account import Credentials
import json
from shapely import wkt
from dotenv import load_dotenv
import sys
from geoalchemy2 import WKTElement
from types import SimpleNamespace
from tqdm import tqdm
from IPython.display import clear_output
import random
from scipy.spatial import cKDTree
import scipy.interpolate
import scipy.spatial.distance
import centerline.geometry
import math
import time


sys.path.append(r"C:\Users\ebeva\SkyTruth\git\cerulean-cloud")
load_dotenv(r"C:\Users\ebeva\.env")

from cerulean_cloud.cloud_function_ais_analysis.utils.analyzer import (  # noqa: E402
    ASA_MAPPING,
    InfrastructureAnalyzer,
    AISAnalyzer,
    SourceAnalyzer,
    DarkAnalyzer,
)

from cerulean_cloud.cloud_function_ais_analysis.utils.constants import (
    AIS_BUFFER,
    AIS_PROJECT_ID,
    AIS_REF_DIST,
    BUF_VEC,
    CLOSING_BUFFER,
    D_FORMAT,
    DECAY_FACTOR,
    HOURS_AFTER,
    HOURS_BEFORE,
    INFRA_MEAN,
    INFRA_REF_DIST,
    INFRA_STD,
    MIN_AREA_THRESHOLD,
    NUM_TIMESTEPS,
    NUM_VERTICES,
    T_FORMAT,
    VESSEL_MEAN,
    VESSEL_STD,
    W_DISTANCE,
    W_OVERLAP,
    W_TEMPORAL,
    WEIGHT_VEC,
)

In [ ]:
def generate_infrastructure_points(
    slick_gdf, num_points, expansion_factor=0.2, crs="epsg:4326"
):
    """
    Generates random infrastructure points within an expanded bounding box of the combined geometry.

    Parameters:
    - slick_gdf (GeoDataFrame): GeoDataFrame containing slick polygons.
    - num_points (int): Number of infrastructure points to generate.
    - expansion_factor (float): Fraction to expand the bounding box.

    Returns:
    - infra_gdf (GeoDataFrame): GeoDataFrame of infrastructure points.
    """
    minx, miny, maxx, maxy = slick_gdf.total_bounds
    width = maxx - minx
    height = maxy - miny
    infra_x = np.random.uniform(
        minx - expansion_factor * width, maxx + expansion_factor * width, num_points
    )
    infra_y = np.random.uniform(
        miny - expansion_factor * height, maxy + expansion_factor * height, num_points
    )
    df = pd.DataFrame(
        {
            "structure_start_date": [pd.Timestamp(0)] * num_points,
            "structure_end_date": [pd.Timestamp.now()] * num_points,
        }
    )
    infra_gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(infra_x, infra_y), crs=crs
    )
    return infra_gdf


def plot_coincidence(
    analyzer,
    slick_id,
    black=True,
    true_point=None,
    nearby_vess=None, 
    padding_ratio = 0.2,
    title = None,
):
    """
    Plots a sample of infrastructure points with their coincidence scores.

    Parameters:
    - analyzer (SourceAnalyzer): Analyzer object containing infrastructure points and coincidence scores.
    - slick_id (int): Identifier for the plot title.
    - black (bool): Whether to use black borders for the infrastructure points.
    """

    sample_size = len(analyzer.infra_gdf)
    plt.figure(figsize=(10, 10))

    # Create an axes object
    ax = plt.gca()

    # First plot the infrastructure points
    scatter = ax.scatter(
        analyzer.infra_gdf.geometry.x[:sample_size],
        analyzer.infra_gdf.geometry.y[:sample_size],
        c=analyzer.coincidence_scores[:sample_size],
        cmap="Blues",
        s=10,
        vmin=0,
        vmax=1,
        # alpha=analyzer.coincidence_scores[:sample_size],
        edgecolor="black" if black else None,  # Adds black borders
        # linewidth=0.5,  # Optional: adjust border thickness
        label="Coincidence Score Distribution",
    )

    # Then plot the slick_gdf polygons on top
    analyzer.slick_gdf.plot(
        edgecolor="red", linewidth=1, color="none", ax=ax, label="Slick Polygons"
    )

    if not true_point is None:
        true_point.plot(
            ax=ax, 
            color="none",  # Ensure no fill color
            edgecolor="none",  # No edge color
        )

        # Add a green X marker using matplotlib
        ax.scatter(
            true_point.geometry.x, 
            true_point.geometry.y, 
            edgecolor='green',
            color="none", 
            alpha=1.0,
            marker="o", 
            label="True Source"
        )
    if not nearby_vess is None:
        nearby_vess.plot(
            ax=ax, 
            color="none",  # Ensure no fill color
            edgecolor="none",  # No edge color
        )

        # Add a red X marker using matplotlib
        ax.scatter(
            nearby_vess.geometry.x, 
            nearby_vess.geometry.y, 
            edgecolor='red',
            color="yellow",
            alpha=0.5,
            marker="x", 
            label="Nearby Vessel"
        )

    # Optionally, plot the centroid on top
    centroid = analyzer.slick_gdf.centroid.iloc[0]
    ax.plot(centroid.x, centroid.y, "k+", markersize=10, label="Centroid")

    # Set plot limits with padding
    min_x, min_y, max_x, max_y = analyzer.slick_gdf.total_bounds
    padding_ratio = padding_ratio

    width = max_x - min_x
    height = max_y - min_y

    padding_x = width * padding_ratio
    padding_y = height * padding_ratio

    # Apply padding
    min_x_padded = min_x - padding_x
    max_x_padded = max_x + padding_x
    min_y_padded = min_y - padding_y
    max_y_padded = max_y + padding_y

    # Determine the larger dimension
    width_padded = max_x_padded - min_x_padded
    height_padded = max_y_padded - min_y_padded

    if width_padded > height_padded:
        # Width is the larger dimension
        extra_height = width_padded - height_padded
        min_y_final = min_y_padded - extra_height / 2
        max_y_final = max_y_padded + extra_height / 2
        min_x_final = min_x_padded
        max_x_final = max_x_padded
    else:
        # Height is the larger dimension
        extra_width = height_padded - width_padded
        min_x_final = min_x_padded - extra_width / 2
        max_x_final = max_x_padded + extra_width / 2
        min_y_final = min_y_padded
        max_y_final = max_y_padded

    ax.set_xlim(min_x_final, max_x_final)
    ax.set_ylim(min_y_final, max_y_final)

    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax)
    cbar.set_label("Coincidence")

    max_coincidence = (
        round(analyzer.coincidence_scores.max(), 2)
        if len(analyzer.coincidence_scores)
        else 0
    )

    # Set titles and labels
    if title is None:
        plt.title(f"Slick ID {slick_id}: Max Coincidence {max_coincidence}")
    else:
        plt.title(f"Slick ID {slick_id}: {title}")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    # Remove or adjust the aspect ratio
    # plt.axis("equal")  # Removed to prevent overriding limits

    # Add grid
    plt.grid(True)

    # Optionally, add a legend
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        plt.legend(handles=handles, labels=labels)

    # Show the plot
    plt.show()

In [ ]:
import math
def SLWBEAR(curve, slick_clean):
    L = curve['length'].values
    A = slick_clean['areas'].values
    return np.sum(L**3 / A) / np.sum(L)

def ARF(curve, slick_clean,ar_ref=16):
    slwbear = SLWBEAR(curve,slick_clean)
    return 1-math.exp((1-slwbear)/ar_ref)

In [ ]:
def get_s1_scene(scene_id, download_path=os.getenv("ASA_DOWNLOAD_PATH")):
    """
    Downloads a S1 scene GeoJSON file from the specified URL if it hasn't been downloaded already.
    """
    url = f"https://api.cerulean.skytruth.org/collections/public.sentinel1_grd/items?scene_id={scene_id}&f=geojson"
    geojson_file_path = os.path.join(download_path, f"{scene_id}.geojson")
    if not os.path.exists(geojson_file_path):
        print(f"Downloading GeoJSON file for Scene {scene_id}...")
        os.system(f'curl "{url}" -o "{geojson_file_path}"')
        print(f"Downloaded GeoJSON to {geojson_file_path}")
    else:
        print(f"GeoJSON file already exists at {geojson_file_path}. Skipping download.")
    s1_gdf = gpd.read_file(geojson_file_path)
    s1_scene = SimpleNamespace(
        scene_id=scene_id,
        scihub_ingestion_time=s1_gdf.scihub_ingestion_time.iloc[0],
        start_time=s1_gdf.start_time.iloc[0],
        end_time=s1_gdf.end_time.iloc[0],
        geometry=WKTElement(str(s1_gdf.geometry.iloc[0])),
    )
    return s1_scene

def download_geojson(id, download_path=os.getenv("ASA_DOWNLOAD_PATH")):
    """
    Downloads a GeoJSON file from the specified URL if it hasn't been downloaded already.

    Parameters:
    - id (int): The unique identifier for the GeoJSON item.
    - download_path (str): The directory path where the GeoJSON will be saved.

    Returns:
    - geojson_file_path (str): The file path to the downloaded GeoJSON.
    """
    url = f"https://api.cerulean.skytruth.org/collections/public.slick/items?id={id}&f=geojson"
    geojson_file_path = os.path.join(download_path, f"{id}.geojson")

    if not os.path.exists(geojson_file_path):
        print(f"Downloading GeoJSON file for ID {id}...")
        os.system(f'curl "{url}" -o "{geojson_file_path}"')
        print(f"Downloaded GeoJSON to {geojson_file_path}")
    else:
        print(f"GeoJSON file already exists at {geojson_file_path}. Skipping download.")

    return geojson_file_path

In [ ]:
def plot_3_gdfs(gdf1,gdf2,gdf3):
    fig, ax = plt.subplots(figsize=(10, 10))
    if not isinstance(gdf1,type(None)):
        gdf1.plot(ax=ax, color="blue", alpha=0.5, edgecolor="black")
    if not isinstance(gdf2,type(None)):
        gdf2.plot(ax=ax, color="none", edgecolor="red", linestyle="--")
    if not isinstance(gdf3,type(None)):
        gdf3.plot(ax=ax, color="green", alpha=1.0, edgecolor="black")

    plt.legend()
    plt.title("Geometry with Bounding Boxes")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.show()



In [ ]:
def compute_dark_vessel_coincidence_and_arf(scene_id, slick_id, sar_detections_gdf, 
                                    cutoff_radius=3000, theta_decay=0, 
                                    decay_factor=4.0, num_vertices=10, 
                                    ):

    geojson_file_path = download_geojson(slick_id)
    slick_gdf = gpd.read_file(geojson_file_path)
    s1_scene = get_s1_scene(scene_id)
    
    dark_analyzer = DarkAnalyzer(
        s1_scene, 
        dark_vessels_gdf=sar_detections_gdf, 
        cutoff_radius=cutoff_radius, 
        theta_decay=theta_decay,
        decay_factor=decay_factor,
        num_vertices=num_vertices,
    )
    return dark_analyzer.compute_coincidence_scores(slick_gdf), dark_analyzer.compute_ARF(slick_gdf)

In [ ]:
def label_coin_with_groundtruth(coin, slick_id, dark_vess_gdf):
    if coin is None:
        return None
    dark_vess = dark_vess_gdf[dark_vess_gdf['slick_id']==slick_id]
    coin = coin.sort_values(by='coincidence_score',ascending=False)
    coin['rank'] = list(range(1,len(coin)+1))
    distances = coin.distance(dark_vess.geometry.iloc[0])
    coin['truth'] = distances <= .005
    return coin

In [ ]:
def compute_coin_on_true_vess_slicks(dark_vess_gdf, sar_detections_gdf,
                                    cutoff_radius=3000, theta_decay=0, 
                                    decay_factor=4.0, num_vertices=10, 
                                     ):
    false_vess_arf = []
    false_vess_coin = []
    true_vess_arf = []
    true_vess_coin = [] 

    true_vess_rank = []
    true_vess_mmsi = []

    for i in tqdm(range(len(dark_vess_gdf))):
        slick_id = dark_vess_gdf['slick_id'].values[i]
        scene_id = dark_vess_gdf['scene_id'].values[i]

        coin, arf = compute_dark_vessel_coincidence_and_arf(scene_id, slick_id, sar_detections_gdf,
                                                            cutoff_radius=cutoff_radius, theta_decay=theta_decay, 
                                                            decay_factor=decay_factor, num_vertices=num_vertices, 
                                                            )
        coin = label_coin_with_groundtruth(coin, slick_id, dark_vess_gdf)

        if coin is None:
            continue

        for _,c in coin.iterrows():
            if c['truth']:
                true_vess_arf.append(arf)
                true_vess_coin.append(c['coincidence_score'])
                true_vess_rank.append(c['rank'])
                true_vess_mmsi.append(c['ssvid'])
            else:
                false_vess_arf.append(arf)
                false_vess_coin.append(c['coincidence_score'])
        clear_output()
    return false_vess_arf, false_vess_coin, true_vess_arf, true_vess_coin , true_vess_rank, true_vess_mmsi

def compute_coin_on_false_positive_slicks(reviewed_fp, fp_sar_detections_gdf,
                                            cutoff_radius=3000, theta_decay=0, 
                                            decay_factor=4.0, num_vertices=10, 
                                          ):
    fp_arf = []
    fp_coin = []
    for i in tqdm(range(len(reviewed_fp))):
        slick_id = reviewed_fp['Slick ID'].values[i]
        scene_id = reviewed_fp['Scene ID'].values[i]
        coin, arf = compute_dark_vessel_coincidence_and_arf(scene_id, slick_id, fp_sar_detections_gdf,
                                                            cutoff_radius=cutoff_radius, theta_decay=theta_decay, 
                                                            decay_factor=decay_factor, num_vertices=num_vertices, 
                                                            )
        if coin is None:
            continue
        for _,c in coin.iterrows():
            fp_arf.append(arf)
            fp_coin.append(c['coincidence_score'])
        clear_output()
    return fp_arf, fp_coin



Load Dark Vessel Data

In [33]:
dark_vess_df = pd.read_csv(r'refined_dark_vessel_dataset.csv').drop(columns='Unnamed: 0').drop(columns='index')
sar_detections = pd.read_csv(r'sar_detections_hitl_dark_ds.csv').drop(columns='Unnamed: 0').drop(columns='index')

true_dark_vess_df = dark_vess_df[dark_vess_df['mmsi'].isna()]

#Toggle this on and off to incorporate long distance coincidence (vessels ~20km or more away from slick)
long_distance_coincidence = [3581643, 3581482, 3581103, 3581287, 3581532, 3581538, 3582446, 3581900, 3582053, 3580996, 3581711, 3581920, 3581075, 3581141, 3581812, 3582235, 3582465, 3582774, 3582584]
dark_vess_df = dark_vess_df[[not (slick_id in long_distance_coincidence) for slick_id in dark_vess_df['slick_id'].values]]

dark_vess_gdf = gpd.GeoDataFrame(
    dark_vess_df, 
    geometry=gpd.points_from_xy(dark_vess_df['lon'], dark_vess_df['lat']),  # Create geometry column
    crs="EPSG:4326"  # Set CRS to WGS 84
)

sar_detections_gdf = gpd.GeoDataFrame(
    sar_detections, 
    geometry=gpd.points_from_xy(sar_detections['detect_lon'], sar_detections['detect_lat']),  # Create geometry column
    crs="EPSG:4326"  # Set CRS to WGS 84
)

sar_detections_gdf = sar_detections_gdf[sar_detections_gdf['structure_id'].isna()]
sar_detections_gdf = sar_detections_gdf.reset_index()

Load False Positives data

In [39]:
reviewed_fp = pd.read_csv(r'HITL Source Dataset ASA - False Positives.csv')
reviewed_fp = reviewed_fp[reviewed_fp['Marked As False']=='y']
fp_sar_ds = pd.read_csv('false_positives_sar_detections.csv').drop(columns=['Unnamed: 0'])

fp_sar_detections_gdf = gpd.GeoDataFrame(
    fp_sar_ds, 
    geometry=gpd.points_from_xy(fp_sar_ds['detect_lon'], fp_sar_ds['detect_lat']),  # Create geometry column
    crs="EPSG:4326"  # Set CRS to WGS 84
)

fp_sar_detections_gdf = fp_sar_detections_gdf[fp_sar_detections_gdf['structure_id'].isna()]
fp_sar_detections_gdf = fp_sar_detections_gdf.reset_index()

Select parameters and compute ARF and coincidence score

In [ ]:
cutoff_radius=20000
theta_decay=2.0
decay_factor=4.0
num_vertices=2


In [ ]:
fp_arf, fp_coin = compute_coin_on_false_positive_slicks(reviewed_fp, fp_sar_detections_gdf,
                                                        cutoff_radius=cutoff_radius,
                                                        theta_decay=theta_decay,
                                                        decay_factor=decay_factor,
                                                        num_vertices=num_vertices,
                                                        )


In [ ]:
false_vess_arf, false_vess_coin, true_vess_arf, true_vess_coin , true_vess_rank, true_vess_mmsi = compute_coin_on_true_vess_slicks(dark_vess_gdf, sar_detections_gdf,                                                                                                                                                        
                                                                                                  cutoff_radius=cutoff_radius,
                                                                                                  theta_decay=theta_decay,
                                                                                                  decay_factor=decay_factor,
                                                                                                  num_vertices=num_vertices,
                                                                                                  )

Hit Rate Metrics for ground truth

In [ ]:
print(str(100*round(len(true_vess_coin)/len(dark_vess_gdf),5))+"%", f"of groundtruth found at {cutoff_radius} meter radius of interest")

In [ ]:
top_1_rate = sum(np.array(true_vess_rank)<=1)/len(dark_vess_gdf)
top_3_rate = sum(np.array(true_vess_rank)<=3)/len(dark_vess_gdf)
print(str(100*round(top_1_rate, 5))+"%", f"top 1 source rate at {cutoff_radius} meter radius of interest")
print(str(100*round(top_3_rate, 5))+"%", f"top 3 source rate at {cutoff_radius} meter radius of interest")

Coincidence Score and ARF for true versus false slick attributions

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for different categories
plt.scatter(false_vess_coin, false_vess_arf, color='yellow', label='False Attributions to Dark Vess Slicks')
plt.scatter(true_vess_coin, true_vess_arf, color='green', label='True Attributions to Dark Vess Slicks')
plt.scatter(fp_coin, fp_arf, color='red', label='False Attributions to False Positive Slicks')

# Labels and title
plt.xlabel("Coin")
plt.ylabel("ARF")
plt.title("True and False Dark Vessel Attributions" + f" - {cutoff_radius}m radius")

# Show legend
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))

# Display plot
plt.show()

View dark vessel slicks with distribution

In [ ]:
ex_id = random.randint(0,len(dark_vess_gdf))
# ex_id = 3
dark_vess = dark_vess_gdf.iloc[[ex_id]].reset_index()
slick_id = dark_vess['slick_id'].values[0]
s1_scene_id = dark_vess['scene_id'].values[0]
geojson_file_path = download_geojson(slick_id)
slick_gdf = gpd.read_file(geojson_file_path)
s1_scene = get_s1_scene(s1_scene_id)
fake_infra_gdf = generate_infrastructure_points(slick_gdf, 50000, expansion_factor=1.0)
dist_analyzer = DarkAnalyzer(s1_scene, dark_vessels_gdf = fake_infra_gdf, cutoff_radius=cutoff_radius, num_vertices=num_vertices, decay_factor=decay_factor,theta_decay=theta_decay)
true_analyzer = DarkAnalyzer(s1_scene, dark_vessels_gdf = dark_vess, cutoff_radius=cutoff_radius, num_vertices=num_vertices, decay_factor=decay_factor,theta_decay=theta_decay)
coincidence_scores = dist_analyzer.compute_coincidence_scores(slick_gdf)
truth = true_analyzer.compute_coincidence_scores(slick_gdf)
if truth is None:
    true_coin = 0.0
else:
    true_coin = truth['coincidence_score'].values[0] if len(truth)>0 else 0.0
clear_output()
print(slick_id)
plot_coincidence(dist_analyzer, slick_id, False, true_point=dark_vess, title = f" Radius {cutoff_radius} meters - True Coincidence Score {round(true_coin,3)}", padding_ratio=.5)

View false positives with distribution

In [ ]:
ex_id = random.randint(0,len(reviewed_fp))
fp = reviewed_fp.iloc[[ex_id]].reset_index()
slick_id = fp['Slick ID'].values[0]
s1_scene_id = fp['Scene ID'].values[0]
geojson_file_path = download_geojson(slick_id)
slick_gdf = gpd.read_file(geojson_file_path)
s1_scene = get_s1_scene(s1_scene_id)
fake_infra_gdf = generate_infrastructure_points(slick_gdf, 50000, expansion_factor=1.0)
dist_analyzer = DarkAnalyzer(s1_scene, dark_vessels_gdf = fake_infra_gdf, cutoff_radius=cutoff_radius, num_vertices=num_vertices, decay_factor=decay_factor, theta_decay=theta_decay)
nearby_analyzer = DarkAnalyzer(s1_scene, dark_vessels_gdf = fp_sar_detections_gdf, cutoff_radius=cutoff_radius, num_vertices=num_vertices, decay_factor=decay_factor, theta_decay=theta_decay)
coincidence_scores = dist_analyzer.compute_coincidence_scores(slick_gdf)
nearby = nearby_analyzer.compute_coincidence_scores(slick_gdf)
nearby_coin = 0.0

if not nearby is None:
    pass
    # nearby = nearby[nearby['ssvid'].isna()].reset_index() #toggle AIS off and on
    if len(nearby) == 0:
        nearby=None
    else:
        nearby_coin = nearby.sort_values(by='coincidence_score', ascending=False).iloc[0]['coincidence_score']

clear_output()
plot_coincidence(dist_analyzer, slick_id, False, true_point=None, nearby_vess = nearby, title = f" Radius {cutoff_radius} meters, highest coin score - {round(nearby_coin,3)}", padding_ratio=1.0)